# Use Case 6 - Teleconnection explorer

## Problem definition

As part of a project on climatic teleconnection, a student investigates how El Niño-Southern Oscillation (ENSO) relates to monsoon rainfall. A result could be a plot showing the sliding correlation between Indian Summer Monsoon Rainfall (ISMR) and SST anomalies [RD-4]. A more sophisticated version of this task would be to calculate the Multivariate ENSO Index (MEI, [RD-5], [RD-6]). Additionally, also the comparison of the ENSO index with other CCI datasets (e.g. Cloud, Fire) would be interesting.

## Required toolbox features

- Access to and ingestion of ESA CCI SST and Soil Moisture data
- Geometric adjustments
- Spatial and temporal filtering and subsetting for both data sets
- Calculation of anomalies and statistical quantities
- Visual presentation of statistical results and time series
- ENSO index calculation from SST data
- Calculation of the absolute anomaly on the log transformed soil moisture data
- Calculation of the correlation between the two time series with a lag of 30 days
- Generation of a correlation map and export of the correlation data
- Generation of a time series plot of the correlation by the selection of a location in South East Asia
- Saving of the image and the underlying data

# Ingest data, spatial and temporal subsetting, variable selection

To reduce the amount of data to download and later work with, it is beneficial to do spatial and temporal subsetting, as well as variable selection as early in the workflow pipeline as possible. Cate supports these operations at download time, or when opening a dataset.

In [ ]:
from cate.core.ds import DATA_STORE_REGISTRY
import cate.ops as ops
from cate.util import ConsoleMonitor

In [ ]:
monitor = ConsoleMonitor()

In [ ]:
data_store = DATA_STORE_REGISTRY.get_data_store('esa_cci_odp')
local_store = DATA_STORE_REGISTRY.get_data_store('local')

In [ ]:
local_store

Set regions of interest

In [ ]:
s_india = '72, 8, 85, 17' # 'lon_min, lat_min, lon_max, lat_max'
pacific = '-175, -10, -115, 10' # 'lon_min, lat_min, lon_max, lat_max'

Download the desired spatial and temporal subsets and ECVs as separate datasets

In [ ]:
soil_sources = data_store.query('esacci.SOILMOISTURE.day.L3S.SSMV.multi-sensor.multi-platform.COMBINED.02-2.r1')
soil_sources

In [ ]:
# Doesn't work, see Issue 256. The dataset is already downloaded
#soil_sources[0].make_local(local_name='SOIL_2007',
#                           region=s_india,
#                           time_range='2007-01-01, 2007-12-31',
#                           var_names='sm, sm_uncertainty',
#                           monitor=monitor)

In [ ]:
sst_sources = data_store.query('esacci.SST.day.L4.SSTdepth.multi-sensor.multi-platform.OSTIA.1-1.r1')
sst_sources

In [ ]:
# The dataset is already downloaded
#sst_sources[0].make_local(local_name='SST_2006_2007',
#                          region=pacific,
#                          time_range='2006-01-01, 2007-12-31',
#                          var_names='analysed_sst, analysis_error',
#                          monitor=monitor)

Open and plot the first time slices of the downloaded datasets

In [ ]:
soil = ops.open_dataset('local.SOIL_2007')

In [ ]:
soil

In [ ]:
%matplotlib inline
ops.plot_map(soil, var='sm', time='2007-01-01', region=s_india, file='/home/ccitbx/Desktop/uc6_fig1.png')

In [ ]:
sst = ops.open_dataset('local.SST_2006_2007')

In [ ]:
sst

In [ ]:
%matplotlib inline
ops.plot_map(sst, var='analysed_sst', time='2006-01-01', region=pacific, file='/home/ccitbx/Desktop/uc6_fig2.png')

# Temporal aggregation, Long Term Averaging

Further operations (index calculation, long term averaging) work on monthly datasets. Hence, aggregate the daily datasets to monthly temporal resolution

In [ ]:
sst_monthly = ops.temporal_aggregation(sst)

In [ ]:
sst_monthly

In [ ]:
soil_monthly = ops.temporal_aggregation(soil)

In [ ]:
soil_monthly

Create a long term averaged sea surface temperature dataset

In [ ]:
sst_lta = ops.long_term_average(sst_monthly)

In [ ]:
sst_lta

The index calculation operation reads the reference dataset from a given file name, hence, save the long term average dataset. This takes longer than previous operations, as most of the actual calculation is done in a streaming fashion, when writing the dataset.

In [ ]:
ops.write_netcdf4(sst_lta, '/home/ccitbx/Desktop/sst_lta.nc')

# ENSO index calculation and correlation analysis

Calculate ENSO index

In [ ]:
enso_index = ops.enso_nino34(ds=sst_monthly,
                             var='analysed_sst',
                             file='/home/ccitbx/Desktop/sst_lta.nc')

In [ ]:
enso_index

ENSO index result is in a tabular format, correlation takes a dataset, hence we convert the table to a dataset

In [ ]:
enso_ds = ops.from_dataframe(enso_index)

In [ ]:
enso_ds

We want to calculate correlation with a one month lag. Subset the datasets accordingly

In [ ]:
soil_monthly_jan_nov = ops.subset_temporal(soil_monthly, '2007-01-01, 2007-11-01')

In [ ]:
soil_monthly_jan_nov.time

In [ ]:
enso_index_dec_oct = ops.subset_temporal(enso_ds, '2006-12-01, 2007-10-01')

In [ ]:
enso_index_dec_oct.time

Extract a point of Soil Moisture data to correlate with the ENSO index

In [ ]:
soil_monthly_jan_nov_point = ops.tseries_point(soil_monthly_jan_nov, (78, 12), 'sm')

In [ ]:
soil_monthly_jan_nov_point

Perform correlation

In [ ]:
corr = ops.pearson_correlation_scalar(enso_index_dec_oct, soil_monthly_jan_nov_point, 'ENSO N3.4 Index', 'sm')

In [ ]:
corr